In [2]:
import numpy as np
import pandas as pd

train_file = "C:/Users/mirni/Documents/Проект Лизы/set1_new.tsv"
test_file = "C:/Users/mirni/Documents/Проект Лизы/set2.tsv"

train_df = pd.read_csv(train_file, sep="\t")
test_df = pd.read_csv(test_file, sep="\t")
print("=== Первые 10 строк из конечной версии файла ===")
print(train_df.head(10))

=== Первые 10 строк из конечной версии файла ===
                                    pID Gender  Age  Value    City TestName  \
0  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48    2.8  Москва     THol   
1  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48    1.6  Москва      LDL   
2  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   30.7  Москва      ALT   
3  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   12.0  Москва      LPA   
4  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   44.4  Москва      ALT   
5  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   24.5  Москва      AST   
6  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   30.5  Москва      AST   
7  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   10.8  Москва     TBil   
8  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48    9.0  Москва      LPA   
9  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48    2.1  Москва      LDL   

  TestUnits        Date  
0      mM/L  2023-04-27  
1      mM/L  2023-04-27  
2  

In [3]:
import pandas as pd

# приводим даты к нормальному виду
train_df["ResDate"] = pd.to_datetime(train_df["Date"], dayfirst=True).dt.date

# базовые фичи на (PatientsGuid, ResDate)
base = (train_df
        .sort_values("Date")
        .groupby(["pID", "Date"], as_index=True)
        .agg({
            "Gender": "first",
            "Age": "first",
            "City": "first"
        }))

# “широкая” таблица: по одной колонке на TestName, значение = Value
vals_wide = (train_df.pivot_table(index=["pID", "Date"],
                            columns="TestName",
                            values="Value",
                            aggfunc="first"))

# объединяем
wide = (base
        .join(vals_wide)
        .reset_index())

print(wide.head(10))


C:\Users\mirni\AppData\Local\Temp\ipykernel_4864\2084886764.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  train_df["ResDate"] = pd.to_datetime(train_df["Date"], dayfirst=True).dt.date


                                    pID        Date Gender  Age         City  \
0  00004638-FF3C-4138-B20F-CCCFB20DF046  2025-04-24      Ж   49       Москва   
1  00008699-11A6-4A43-A073-D7A29B7D9ADA  2023-06-21      Ж   32  Красногорск   
2  00008699-11A6-4A43-A073-D7A29B7D9ADA  2024-01-26      Ж   33       Москва   
3  00008699-11A6-4A43-A073-D7A29B7D9ADA  2024-08-18      Ж   33       Москва   
4  0000E88C-1F93-46AE-98D0-19DE1B5EAA83  2025-05-25      Ж   41  Красногорск   
5  00020D84-9344-45C2-89CE-569603A01FC6  2024-10-17      Ж   50       Москва   
6  0003D22A-0A53-4402-9A88-7407A0327796  2023-07-13      Ж   41       Москва   
7  0003F621-73A7-40D3-801B-C9D14ACF7FEB  2024-05-11      Ж   52        Пермь   
8  00048479-A156-4CCA-B9DF-6B7BFD1598CC  2023-11-03      Ж   66  Красногорск   
9  000562FD-AB7B-43E0-8161-10FBD25C6CEC  2022-07-29      Ж   34  Красногорск   

    ALT   AST  DBil   HDL  LDL   LPA  TBil  THol  Trig  
0  38.5  23.0   NaN  1.80  4.1   9.0  11.8   6.8  1.97  
1   9

In [4]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error


# ========= 1) ХЕЛПЕР: разбиение по пациентам =========
def group_train_val_test_split(df, group_col, target_col,
                               train_size=0.70, val_size=0.10, test_size=0.20,
                               random_state=42):
    assert abs(train_size + val_size + test_size - 1.0) < 1e-9, "Доли должны суммироваться в 1"
    groups = df[group_col]

    # 1) Train vs Temp
    gss1 = GroupShuffleSplit(n_splits=1, train_size=train_size, random_state=random_state)
    train_idx, temp_idx = next(gss1.split(df, groups=groups))
    df_train, df_temp = df.iloc[train_idx], df.iloc[temp_idx]

    # 2) Temp -> Val vs Test
    gss2 = GroupShuffleSplit(n_splits=1,
                             test_size=test_size/(val_size+test_size),
                             random_state=random_state)
    val_idx_rel, test_idx_rel = next(gss2.split(df_temp, groups=df_temp[group_col]))
    df_val, df_test = df_temp.iloc[val_idx_rel], df_temp.iloc[test_idx_rel]

    # 3) Разделяем на X/y
    X_train, y_train = df_train.drop(columns=[target_col]), df_train[target_col]
    X_val,   y_val   = df_val.drop(columns=[target_col]),   df_val[target_col]
    X_test,  y_test  = df_test.drop(columns=[target_col]),  df_test[target_col]

    # 4) Проверка непересекаемости групп
    g_train = set(df_train[group_col].unique())
    g_val   = set(df_val[group_col].unique())
    g_test  = set(df_test[group_col].unique())

    assert g_train.isdisjoint(g_val),  "Train и Val пересекаются по patient_id!"
    assert g_train.isdisjoint(g_test), "Train и Test пересекаются по patient_id!"
    assert g_val.isdisjoint(g_test),   "Val и Test пересекаются по patient_id!"

    return X_train, X_val, X_test, y_train, y_val, y_test





In [57]:

# ==== обрезка по квантилям AST ====
data_quant = wide.copy()

columns_to_control = ["ALT", "AST", "TBil"]
for col in columns_to_control:
  q1 = data_quant[col].quantile(0.15)
  q2 = data_quant[col].quantile(0.75)
  data_quant = data_quant[(data_quant[col] >= q2)]
  print(f"Оставляем {col} в пределах:", q2, "-", np.max(data_quant[col]))
print(len(data_quant))

Оставляем ALT в пределах: 30.2 - 3343.4
Оставляем AST в пределах: 38.9 - 3391.2
Оставляем TBil в пределах: 18.3 - 563.1
518


Отрезаем по 75 квантилю, так как если отрезать по 85, то будет всего 121 с

In [58]:
data_quant.describe()

,Age,ALT,AST,DBil,HDL,LDL,LPA,TBil,THol,Trig
count,518.000000,518.000000,518.000000,256.000000,518.000000,518.000000,518.000000,518.000000,510.000000,518.000000
mean,52.297297,139.993050,117.614286,16.498047,1.305618,3.452104,26.519305,35.870077,5.361961,1.714286
std,15.729103,282.948438,280.305393,32.373524,0.500216,1.345829,39.349825,44.278510,1.837613,1.050361
min,17.000000,30.200000,38.900000,1.800000,0.210000,0.700000,0.000000,18.300000,1.400000,0.300000
25%,41.000000,51.125000,44.500000,3.900000,0.992500,2.500000,5.250000,20.425000,4.100000,1.000000
50%,51.000000,73.000000,57.950000,5.600000,1.240000,3.300000,10.000000,24.400000,5.200000,1.450000
75%,63.000000,112.050000,96.700000,10.900000,1.540000,4.200000,29.000000,32.900000,6.400000,2.060000
max,98.000000,3343.400000,3391.200000,267.700000,3.720000,11.000000,341.000000,563.100000,15.400000,7.610000


In [59]:
target = "LPA"

# пол: M -> 0, Ж -> 1
data_quant["Gender"] = data_quant["Gender"].map({"M": 0, "Ж": 1})

# убираем неинформативные/служебные колонки
drop_cols = ["Date", "City"]
data_quant = data_quant.drop(columns=drop_cols)

In [60]:
# делим на Train / Val / Test

X_train, X_val, X_test, y_train, y_val, y_test = group_train_val_test_split(
    data_quant, group_col="pID", target_col=target,
    train_size=0.70, val_size=0.10, test_size=0.20, random_state=42
)

# после сплита удаляем PatientsGuid из признаков
for X in (X_train, X_val, X_test):
    if "pID" in X.columns:
        X.drop(columns=["pID"], inplace=True)

In [61]:
# 1) PREP: один раз считаем импутацию/скейл
from sklearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler

# Если KNNImputer медленный, можно быстро переключиться на SimpleImputer(strategy="median")
imp_only   = KNNImputer(n_neighbors=3, weights="uniform")
imp_scale  = make_pipeline(KNNImputer(n_neighbors=3, weights="uniform"), StandardScaler())

Xtr_imp_quant   = imp_only.fit_transform(X_train)     # для деревьев/бустингов
Xval_imp_quant  = imp_only.transform(X_val)
Xtr_sc_quant    = imp_scale.fit_transform(X_train)    # для линейных/MLP/KNN
Xval_sc_quant   = imp_scale.transform(X_val)

In [62]:

def rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))
# 2) Бейзлайн
baseline_val = np.full_like(y_val, fill_value=np.nanmean(y_train), dtype=float)
results = [("BaselineMean", rmse(y_val, baseline_val))]

print(f"Результат предсказания средним {rmse(y_val, baseline_val)}")

Результат предсказания средним 40.58374361198614


In [63]:
# 3) МОДЕЛИ (без пайплайнов — мы уже подготовили матрицы)
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor, callback
from lightgbm import LGBMRegressor
import lightgbm as lgb

# ЛИНЕЙНЫЕ/MLP/KNN — обучаем на Xtr_sc_quant
lin = LinearRegression()
lin.fit(Xtr_sc_quant, y_train)
results.append(("LinearRegression", rmse(y_val, lin.predict(Xval_sc_quant))))

ridge = RidgeCV(alphas=np.logspace(-3, 3, 25), cv=5)
ridge.fit(Xtr_sc_quant, y_train)
results.append(("RidgeCV", rmse(y_val, ridge.predict(Xval_sc_quant))))

lasso = LassoCV(alphas=np.logspace(-3, 1, 20), cv=5, max_iter=10000, n_jobs=-1)
lasso.fit(Xtr_sc_quant, y_train)
results.append(("LassoCV", rmse(y_val, lasso.predict(Xval_sc_quant))))

knn = KNeighborsRegressor(n_neighbors=15, weights="distance")
knn.fit(Xtr_sc_quant, y_train)
results.append(("KNN(k=15)", rmse(y_val, knn.predict(Xval_sc_quant))))

mlp = MLPRegressor(hidden_layer_sizes=(128,64), activation="relu",
                   alpha=1e-4, batch_size=256, learning_rate_init=1e-3,
                   max_iter=200, random_state=42, early_stopping=True, n_iter_no_change=10)
mlp.fit(Xtr_sc_quant, y_train)
results.append(("MLP", rmse(y_val, mlp.predict(Xval_sc_quant))))

# ДЕРЕВЬЯ/БУСТИНГИ — обучаем на Xtr_imp с ранней остановкой ОДИН РАЗ
rf = RandomForestRegressor(
    n_estimators=400, max_depth=None, min_samples_leaf=2,
    max_features="sqrt", n_jobs=-1, random_state=42
)
rf.fit(Xtr_imp_quant, y_train)
results.append(("RandomForest", rmse(y_val, rf.predict(Xval_imp_quant))))


In [69]:
def fit_pick_best(model_ctor, param_grid, Xtr, ytr, Xval, yval, name):
    best_model, best_rmse, best_params = None, float("inf"), None
    for params in param_grid:
        model = model_ctor(**params)
        model.fit(Xtr, ytr)
        pred = model.predict(Xval)
        score = rmse(yval, pred)
        # print(f"{name} {params} -> RMSE {score:.4f}")  # при желании включи
        if score < best_rmse:
            best_rmse, best_model, best_params = score, model, params
    # print(f"BEST {name}: {best_params} -> {best_rmse:.4f}")
    return best_model, best_rmse

# ---------- XGBoost (без ES), быстрый грубый подбор ----------
xgb_grid = [
    dict(
        n_estimators=ne,
        learning_rate=lr,
        max_depth=md,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.0,
        reg_lambda=1.0,
        n_jobs=-1,
        random_state=42,
        tree_method="hist",   # быстро
        max_bin=256
    )
    for ne in (300, 600)          # маленький потолок
    for lr in (0.07, 0.10)        # быстрее с чуть бОльшим шагом
    for md in (4, 6)              # неглубокие деревья — быстрее/стабильнее
]

xgb_model, xgb_rmse = fit_pick_best(XGBRegressor, xgb_grid, Xtr_imp_quant, y_train, Xval_imp_quant, y_val, "XGB")
results.append(("XGBoost", xgb_rmse))

#---------- LightGBM (без ES), быстрый грубый подбор ----------
# lgb_grid = [
#     dict(
#         n_estimators=ne,
#         learning_rate=lr,
#         num_leaves=nl,          # связана с max_depth≈log2(nl)
#         max_depth=-1,
#         min_child_samples=2,
#         subsample=0.8,
#         subsample_freq=1,
#         colsample_bytree=0.8,
#         reg_alpha=0.0,
#         reg_lambda=1.0,
#         random_state=42,
#         n_jobs=-1,
#         boosting_type="gbdt",
#         max_bin=255,
#         # device_type="cpu"  # если ставили GPU, можно явно указать cpu
#     )
#     for ne in (400, 800)
#     for lr in (0.05, 0.08)
#     for nl in (31, 63)
# ]

# lgb_model, lgb_rmse = fit_pick_best(LGBMRegressor, lgb_grid, Xtr_imp_quant, y_train, Xval_imp_quant, y_val, "LGBM")
# results.append(("LightGBM", lgb_rmse))

In [70]:
# ----- 1) Печать валидационных результатов -----
print("\nRMSE на Validation:")
for name, score in sorted(results, key=lambda x: x[1]):
    print(f"{name:>15}: {score:.4f}")

best_name = min(results, key=lambda x: x[1])[0]
print("\nЛучшая модель:", best_name)


RMSE на Validation:
       LightGBM: 39.8377
   RandomForest: 40.1814
LinearRegression: 40.1978
        RidgeCV: 40.2609
        XGBoost: 40.3967
        XGBoost: 40.3967
        XGBoost: 40.3967
   BaselineMean: 40.5837
        LassoCV: 40.5837
      KNN(k=15): 41.2216
            MLP: 42.1178

Лучшая модель: LightGBM
